In [1]:
import numpy as np
## Define the tree with one vertex and weight 1
one_tree = LabelledRootedTree([], label = (1,0))
## Define the tree with one vertex and weight 0
zero_tree = LabelledRootedTree([], label = (0,0))

def bare_trees(k): 
    result = [set() for i in np.arange(k+1)]
    result[0].add(zero_tree)
    if k == 0:
        return result[0]
    else:
        for i in range(1,k+1):
            for t in result[i-1]:
                for p in list(OrderedTree(t).paths()):
                    ## add a vertex to any vertex in the tree
                    result[i].add(t.single_graft(LabelledRootedTree([one_tree]),[p]))
                    ## increase the weight of a node by 1
                    temp = None
                    with t.clone() as tc:
                        tc.set_label(p, (t.label(p)[0]+1,0))
                        temp = tc ## Need to add outside of the clone thing, otherwise there is a bug
                    result[i].add(temp)
        return result[i]

In [2]:
# k is the total weight of the tree
#k = 4
#for i in range(0,k+1):
#    print("There are",len(bare_trees(i)),"bare trees with weight",i)

In [3]:
## Define a function that given a labelled tree and a path to a node, returns the number of children
def children(t,p):
    temp = t.shape()
    for i in p:
        temp = temp[i]
    return(len(temp))
def valency(t,p):
    return(children(t,p)+1+t.label(p)[1])
def weight_at_p(t,p):
    return t.label(p)[0]
def total_weight(t):
    result = 0
    for p in OrderedTree(t).paths():
        result += weight_at_p(t,p)
    return result

In [4]:
## Example
t = LabelledRootedTree([one_tree,one_tree],label = 0)
t = LabelledRootedTree([t,t],label = 0)
t = LabelledRootedTree([t,t],label = 0)
p = list(OrderedTree(t).paths())[2]
#children(t,p) ## should be 2

In [5]:
## Example
t = LabelledRootedTree([one_tree,one_tree,one_tree,one_tree],label = (0,0))
number_leafs = 0
for p in OrderedTree(t).paths():
    if t.label(p)[0] + 2 - children(t,p) >= 0:
        number_leafs += t.label(p)[0] + 2 - children(t,p)
#print("the tree",t,"has a minimum of",number_leafs,"leafs")

In [6]:
## Given a number legs n and a weight k, returns a list of possible trees
def possible_trees(k,n):
    return_trees = [t for t in bare_trees(k)]
    ## filter our the trees that can't have n leafs
    for t in return_trees.copy():
        leafs_t = 0
        for p in list(OrderedTree(t).paths()):
            if t.label(p)[0] + 3 - valency(t,p) >= 0:
                requiered_leafs_t_p = t.label(p)[0] + 3 - valency(t,p)
                leafs_t += requiered_leafs_t_p
        if leafs_t > n:
            return_trees.remove(t)
    return return_trees

In [7]:
#n = 10
#k = 4
#for i in np.arange(k+1):
#    print(len(possible_trees(i,n)),"trees of weight",i,"with",n,"leafs are possible")

In [8]:
## Given a bare tree and a number of leafs, returns the tree with the minimum number of leafs on it
def tree_with_minimum_leafs(t,n):
    leafs_minimum_t = 0
    ## add the required amount of leafs as labels to t
    for p in reversed(list(OrderedTree(t).paths())):    #Go backwards through paths, because
                                                        # after chaning label tree gets reordered
        requiered_leafs_t_p = t.label(p)[0] + 3 - valency(t,p)
        if requiered_leafs_t_p > 0:
            with t.clone() as tc:
                tc.set_label(p, (t.label(p)[0],requiered_leafs_t_p))
                temp = tc ## Need to add outside of the clone thing, otherwise there is a bug
            t = temp
            leafs_minimum_t += requiered_leafs_t_p
    return t

In [9]:
### Checking
#for t in bare_trees(4):
#    tc = tree_with_minimum_leafs(t,10)
#    for p in list(OrderedTree(tc).paths()):
#        if not valency(tc,p)-3-weight(tc,p) == 0:
#            print(tc,"at",p,"has",valency(tc,p)-3-weight(tc,p))

In [10]:
## Given a weight k and a number of leafs n, returns a list of trees with minimum leafs
def list_of_trees_with_minimum_leafs(k,n):
    return_trees = []
    for t in possible_trees(k,n):
        return_trees += [tree_with_minimum_leafs(t,n)]
    return return_trees

In [11]:
## Given a tree returns the number of leafs on it
def number_of_leafs(t):
    return_leafs = 0
    for p in list(OrderedTree(t).paths()):
        return_leafs += t.label(p)[1]
    return return_leafs

In [12]:
# an iterator over the points of N^n with coordinates summing to d
# n must be at least 1
def simplex(n,d):
    if n == 0 or d < 0 : raise ValueError("invalid input, n must be positive and d must be nonnegative : (n, d) = (%d, %d)" % (n, d))
    elif n == 1 :
        yield [d]
    else:
        for d_last in range(d+1):
            for p in simplex(n-1, d - d_last):
                yield p + [d_last]

In [13]:
### Testing simplex function
###list(simplex(3, 3))

In [14]:
def set_of_trees_from_t(t,n):
    return_trees = set()
    leafs_left_t = n-number_of_leafs(t)
    if leafs_left_t == 0:
        return {t}
    number_nodes = len(list(OrderedTree(t).paths()))
    for leaf_count in simplex(number_nodes, leafs_left_t):
        # leaf_count is the list of integers saying how many leaves to add to each node on the tree
        temp = None
        with t.clone() as tc:
            for p, l in reversed(list(zip(OrderedTree(t).paths(), leaf_count))):
                tc.set_label(p, (t.label(p)[0], t.label(p)[1]+l))
            temp = tc
        return_trees.add(temp)
    return return_trees

In [15]:
def old_set_of_trees_from_t(t,n):
    return_trees = set()
    leafs_left_t = n-number_of_leafs(t)
    if leafs_left_t == 0:
        return {t}
    number_nodes = len(list(OrderedTree(t).paths()))
    for leaf_count in simplex(number_nodes, leafs_left_t):
        # leaf_count is the list of integers saying how many leaves to add to each node on the tree
        temp = None
        with t.clone() as tc:
            for p, l in zip(OrderedTree(t).paths(), leaf_count):
                tc.set_label(p, (t.label(p)[0], t.label(p)[1]+l))
            temp = tc
        return_trees.add(temp)
    return return_trees

In [16]:
## Given weight k, number of leafs n, returns a list of possible trees
def list_trees(k,n):
    return_trees = []
    for t in list_of_trees_with_minimum_leafs(k,n):
        for s in set_of_trees_from_t(t,n):
            return_trees += [s]
    return return_trees

In [17]:
## Given weight k, number of leafs n, returns a list of possible trees
def old_list_trees(k,n):
    return_trees = []
    for t in list_of_trees_with_minimum_leafs(k,n):
        for s in old_set_of_trees_from_t(t,n):
            return_trees += [s]
    return return_trees

In [18]:
#len(list_trees(4,10))

In [19]:
#for i, t in enumerate(old_list_trees(4,10)):
#    if not number_of_leafs(t) == 10:
#        print(i,t,"does not have the right number of leafs")
#    if not total_weight(t) == 4:
#        print(t,"does not have the right weight")

In [20]:
bad_tree = None
for t in list_of_trees_with_minimum_leafs(4,10):
    for s in old_set_of_trees_from_t(t,10):
        if not number_of_leafs(s) == 10:
            #print(set_of_trees_from_t(t,10))
            bad_tree = s

In [21]:
t = bad_tree
#t

In [22]:
#(1, 2)[(1, 1)[(1, 3)[], (1, 3)[]]]
#list(simplex(4,2))

In [73]:
leafs_left_t = 10-number_of_leafs(t)
number_nodes = len(list(OrderedTree(t).paths()))
#list(simplex(number_nodes, leafs_left_t))

In [76]:
leaf_count = list(simplex(number_nodes, leafs_left_t))[2]
#print("leaf_count:",leaf_count)
temp = None
with t.clone() as tc:
    for p, l in list(zip(OrderedTree(t).paths(), leaf_count)):
        tc.set_label(p, (t.label(p)[0], t.label(p)[1]+l))
        #print(p,tc)

In [77]:
### The resulting tree has only 9 leafs!!!!